# Agent Development Kit - Getting Started

This notebook demonstrates basic usage of the Agent Development Kit (ADK) for experimenting with Google Cloud AI agents.

## Setup and Imports

In [ ]:
import sys
import os

# Add src to path for imports
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from adk import setup_logging
from adk.agents import EchoAgent, VertexAIAgent
from adk.utils import print_environment_status, validate_environment
from adk.config import get_config

# Setup logging for notebook
setup_logging(level="INFO", format_type="console")

print("✅ ADK imported successfully")

## Environment Validation

In [ ]:
# Check environment setup
print_environment_status()

# Get detailed validation results
env_results = validate_environment()
print("\n📊 Detailed Results:")
for key, value in env_results.items():
    print(f"  {key}: {value}")

## Configuration

In [ ]:
# Load configuration
try:
    config = get_config()
    print(f"Project: {config.google_cloud_project}")
    print(f"Location: {config.vertex_ai_location}")
    print(f"Agent Name: {config.agent_name}")
except Exception as e:
    print(f"⚠️ Configuration issue: {e}")
    print("💡 Make sure to create .env file with GOOGLE_CLOUD_PROJECT")

## Echo Agent Example

In [ ]:
# Create and test Echo Agent
echo_agent = EchoAgent(name="NotebookEcho", description="Echo agent for notebook demo")
await echo_agent.initialize()

print("🤖 Echo Agent initialized")
print(f"   Name: {echo_agent.name}")
print(f"   Description: {echo_agent.description}")

In [ ]:
# Test basic message processing
response = await echo_agent.process_message("Hello from Jupyter notebook!")
print(f"Response: {response}")

In [ ]:
# Test with context
context = {
    "user": "notebook_user",
    "session_id": "jupyter_session_123",
    "environment": "development"
}

response = await echo_agent.process_message(
    "This message includes context!", 
    context=context
)
print(f"Response with context: {response}")

In [ ]:
# Health check
health = await echo_agent.health_check()
print(f"Health status: {health}")

## Vertex AI Agent Example

In [ ]:
# Test Vertex AI Agent (requires proper Google Cloud setup)
try:
    vertex_agent = VertexAIAgent(
        name="NotebookVertex", 
        description="Vertex AI agent for notebook demo",
        model_name="text-bison"
    )
    
    await vertex_agent.initialize()
    print("🧠 Vertex AI Agent initialized successfully")
    
except Exception as e:
    print(f"⚠️ Vertex AI Agent initialization failed: {e}")
    print("💡 This is expected if Google Cloud is not configured")
    vertex_agent = None

In [ ]:
# Test Vertex AI if available
if vertex_agent:
    try:
        prompt = "Write a short, creative story about a robot learning to paint."
        response = await vertex_agent.process_message(prompt)
        print(f"Vertex AI Response: {response}")
    except Exception as e:
        print(f"Error calling Vertex AI: {e}")
else:
    print("Vertex AI agent not available - skipping test")

## Interactive Agent Testing

In [ ]:
# Interactive cell for testing agents
async def test_agent_interactive(agent, message):
    """Helper function to test agents interactively"""
    try:
        response = await agent.process_message(message)
        print(f"Agent: {agent.name}")
        print(f"Input: {message}")
        print(f"Output: {response}")
        print("-" * 50)
    except Exception as e:
        print(f"Error: {e}")

# Test different messages
test_messages = [
    "What is artificial intelligence?",
    "Tell me a joke",
    "Explain quantum computing in simple terms"
]

print("🧪 Testing agents with sample messages:\n")

for message in test_messages:
    await test_agent_interactive(echo_agent, message)
    
    if vertex_agent:
        await test_agent_interactive(vertex_agent, message)

## Custom Agent Development

In [ ]:
# Example of creating a custom agent
from adk.agents import BaseAgent
import random

class RandomResponseAgent(BaseAgent):
    """Agent that returns random responses from a predefined list"""
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.responses = [
            "That's an interesting question!",
            "Let me think about that...",
            "Great point! Here's what I think:",
            "I see what you mean.",
            "That reminds me of something..."
        ]
    
    async def initialize(self):
        self.logger.info("Random response agent initialized")
    
    async def process_message(self, message, context=None):
        random_response = random.choice(self.responses)
        return f"{random_response} You said: '{message}'"

# Test custom agent
custom_agent = RandomResponseAgent(name="CustomRandom", description="Random response generator")
await custom_agent.initialize()

print("🎲 Testing custom agent:")
for i in range(3):
    response = await custom_agent.process_message(f"Test message {i+1}")
    print(f"  Response {i+1}: {response}")

## Next Steps

1. **Configure Google Cloud**: Set up your `.env` file with proper credentials
2. **Explore Examples**: Check the `examples/` directory for more advanced usage
3. **Build Custom Agents**: Create your own agents by extending `BaseAgent`
4. **API Server**: Run the FastAPI server for web-based interactions
5. **Production Deploy**: Scale your agents for production use

### Useful Commands
```bash
# Run basic example
python examples/basic_agent_example.py

# Start API server
python examples/server.py

# Run tests
pytest tests/
```